In [1]:
import json
import os
import random
import fnmatch
import openpyxl
from datetime import datetime

from urllib.request import urlopen
import urllib

import plotly.graph_objects as go
import plotly.express as px
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
import numpy as np
import re

In [2]:
sns.set_style("darkgrid")

In [3]:
def import_json_to_dict(url) :
    response = urllib.request.urlopen(url)
    my_dict = json.loads(response.read())
    return my_dict

In [4]:
def clean_str(s):
    d = {
        "’": "'",
        "\xa0": " ",
        "/":","
        }
    for x in d:
        s = s.replace(x, d[x]).strip()
    return s

In [5]:
month_dict = {
    "January" : "Janvier",
"February": "Février",
"March": "Mars",
"April" : "Avril",
"May" : "Mai",
"June" : "Juin",
"July" : "Juillet",
"August" : "Août",
"September":"Septembre",
"October":"Octobre",
"November":"Novembre",
"December":"Décembre"}

In [6]:
taxo_dep_df = pd.read_csv('refs/taxo_deps.csv', dtype={'dep':str, 'reg':str})
taxo_dep_df['dep'] = taxo_dep_df['dep'].apply(lambda x: x.zfill(2))
taxo_dep_df['reg'] = taxo_dep_df['reg'].apply(lambda x: x.zfill(2))
dep_list = list(taxo_dep_df['dep'].unique())
taxo_dep_df

,cheflieu,dep,libelle,ncc,nccenr,reg,tncc
0,1053,01,Ain,AIN,Ain,84,5.0
1,2408,02,Aisne,AISNE,Aisne,32,5.0
2,3190,03,Allier,ALLIER,Allier,84,5.0
3,4070,04,Alpes-de-Haute-Provence,ALPES DE HAUTE PROVENCE,Alpes-de-Haute-Provence,93,4.0
4,5061,05,Hautes-Alpes,HAUTES ALPES,Hautes-Alpes,93,4.0
...,...,...,...,...,...,...,...
105,98700,987,Polynésie française,POLYNESIE FRANCAISE,Polynésie française,COM,NaN
106,98400,984,Terres australes et antarctiques françaises,TERRES AUSTRALES ET ANTARCTIQUES FRANCAISES,Terres australes et antarctiques françaises,984,NaN
107,98800,988,Nouvelle-Calédonie,NOUVELLE-CALEDONIE,Nouvelle-Calédonie,988,NaN
108,98900,989,Île de Clipperton,ILE DE CLIPPERTON,Île de Clipperton,989,NaN


In [7]:
taxo_reg_df = pd.read_csv('refs/taxo_regions.csv', dtype={'reg':str})
taxo_reg_df['reg'] = taxo_reg_df['reg'].apply(lambda x: x.zfill(2))
reg_list = list(taxo_reg_df['reg'].unique())
taxo_reg_df

,cheflieu,libelle,ncc,nccenr,reg,tncc
0,97105,Guadeloupe,GUADELOUPE,Guadeloupe,01,3.0
1,97209,Martinique,MARTINIQUE,Martinique,02,3.0
2,97302,Guyane,GUYANE,Guyane,03,3.0
3,97411,La Réunion,LA REUNION,La Réunion,04,0.0
4,97608,Mayotte,MAYOTTE,Mayotte,06,0.0
5,75056,Île-de-France,ILE DE FRANCE,Île-de-France,11,1.0
6,45234,Centre-Val de Loire,CENTRE VAL DE LOIRE,Centre-Val de Loire,24,2.0
7,21231,Bourgogne-Franche-Comté,BOURGOGNE FRANCHE COMTE,Bourgogne-Franche-Comté,27,0.0
8,76540,Normandie,NORMANDIE,Normandie,28,0.0
9,59350,Hauts-de-France,HAUTS DE FRANCE,Hauts-de-France,32,4.0


In [8]:
data_dir_path = './data/'

In [9]:
mailles_list = ['national', 'regional', 'departemental']

In [10]:
propilot_path = os.path.join("data")

In [11]:
df_dict = {}
file_list = [
"dim_activity",
"dim_tree_nodes",
"dim_top_levels",
"dim_maturities",
"dim_period",
"dim_snapshots",
"dim_effects",
"dim_properties",
"dim_states",
"dim_structures",
"fact_financials",
"fact_property_values"
]

In [12]:
for file in file_list:
    for file_csv in os.listdir(propilot_path):
        if fnmatch.fnmatch(file_csv, file + "*.csv"):
            print("File loaded : ", file_csv)
            df_dict[file] = pd.read_csv(os.path.join(propilot_path, file_csv), sep=";")

File loaded :  dim_activity.csv
File loaded :  dim_tree_nodes.csv
File loaded :  dim_top_levels.csv
File loaded :  dim_maturities.csv
File loaded :  dim_period_pp_ch.csv
File loaded :  dim_snapshots.csv
File loaded :  dim_effects.csv
File loaded :  dim_properties.csv
File loaded :  dim_states_pp_ch.csv
File loaded :  dim_structures.csv
File loaded :  fact_financials_enr_pp_ch.csv


/home/yves/anaconda3/envs/propilot/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3165: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


File loaded :  fact_property_values.csv


In [13]:
df = (df_dict["fact_financials"]
      .merge(df_dict["dim_tree_nodes"], left_on="tree_node_id", right_on="tree_node_id")
      .merge(df_dict["dim_effects"], left_on="effect_id", right_on="effect_id")
      .merge(df_dict["dim_states"], left_on="state_id", right_on="state_id")
      .merge(df_dict["dim_period"], left_on="period_id", right_on="period_id")
      .merge(df_dict["dim_structures"], left_on="structure_id", right_on="structure_id"))

In [14]:
def check_data_merge(facts, df):
    num_pp_recs = facts[facts['financials_source'] == 'proPilot'].shape[0]
    num_df_recs = df.shape[0]
    assert num_pp_recs == num_df_recs, f"Le nombre d'enregistrements proPilot diffèrent : avant {num_pp_recs} - après {num_df_recs}"
    

check_data_merge(df_dict["fact_financials"], df)

In [15]:
def extract_dep_code(expr):
    nums = re.findall(r'D\d+', expr)
    if expr.endswith('D2A'):
        return '2A'
    elif expr.endswith('D2B'):
        return '2B'
    elif expr.endswith('E00'):
        return '00'
    return nums[0][1:].zfill(2) if len(nums) > 0 else None


df['dep_code'] = df['tree_node_code'].apply(lambda x: extract_dep_code(x))

In [16]:
def check_cols(cols, df):
    assert set(cols).issubset(df.columns), f"Colonnes manquantes : {set(cols) - set(df.columns)}"

In [17]:
cols = ["tree_node_name", "structure_name", "effect_id", "state_id", "period_date", "period_month_tri", "period_month_year", "financials_cumulated_amount", "dep_code"]
check_cols(cols, df)

df=df[cols]
df.rename(columns={"period_month_year":"Date", "financials_cumulated_amount":"valeur"}, inplace=True)

In [18]:
forbidden_period_value = ["Y", "Total"]

In [19]:
def clean_mesure_name(tree_node_name):
    raw_mesure = tree_node_name.split('/')[1].strip() if '/' in tree_node_name else tree_node_name
    # nettoyage de la colonne mesure, on enlève un point surnuméraire.
    mesure = re.sub('\.', "", raw_mesure)
    mesure = clean_str(mesure)
    return mesure

In [20]:
def check_single_sum_dep_equals_nat(df_copy, mesure, indic):
    nat_values = df_copy[(df_copy['structure_name'] == 'Mesure') & 
                    (df_copy['mesure'] == mesure) &
                    (df_copy['effect_id'] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['structure_name'] == 'Département') & 
                            (df_copy['mesure'] == mesure) & 
                            (df_copy['effect_id'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')
            

def check_sum_dep_equals_nat(df):
    """ Ce test a été rajouté après avoir constaté des différences entre somme des départements et valeurs nationales."""
    df_copy = df.copy()
    df_copy = df_copy.loc[
                (~df_copy.period_month_tri.isin(forbidden_period_value)) &
                (df_copy.state_id == 'Valeur Actuelle') &
                (~df_copy.valeur.isna())].copy()

    df_copy["mesure"] = df_copy["tree_node_name"].apply(lambda x: clean_mesure_name(x))

    mesures = df_copy['mesure'].unique()
    for mesure in mesures:
        indics = df_copy[df_copy['mesure'] == mesure]['effect_id'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat(df_copy, mesure, indic)

            
check_sum_dep_equals_nat(df)

KO - Repas à 1 € dans les restaurants universitaires - Nombre de repas servis dans les restaurants universitaires au tarif à 1€ - RRU - 2021-01-31T00:00:00.0000000 | 3787789.0 - 3490110.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-01-31T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-02-28T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-03-31T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-01-31T00:00:00.0000000 | 1277884181.0 - 1284896505.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-02-28T00:00:00.0000000 | 1277884181.0 - 1284896505.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-03-31T00:00:00.0000000 | 1277884181.0 - 1284896505.0

In [21]:
df_dep = df.loc[
    (df.structure_name == "Département") & 
    (~df.period_month_tri.isin(forbidden_period_value)) &
    (df.state_id == 'Valeur Actuelle') &
    (~df.valeur.isna())].copy()

In [22]:
df_dep['period_date'].unique()

array(['2021-01-31T00:00:00.0000000', '2021-02-28T00:00:00.0000000',
       '2020-11-30T00:00:00.0000000', '2020-12-31T00:00:00.0000000',
       '2021-03-31T00:00:00.0000000', '2021-04-30T00:00:00.0000000',
       '2021-05-31T00:00:00.0000000', '2021-06-30T00:00:00.0000000',
       '2021-07-31T00:00:00.0000000', '2021-08-31T00:00:00.0000000',
       '2021-09-30T00:00:00.0000000', '2021-10-31T00:00:00.0000000',
       '2021-11-30T00:00:00.0000000', '2021-12-31T00:00:00.0000000',
       '2022-01-31T00:00:00.0000000', '2022-02-28T00:00:00.0000000',
       '2022-03-31T00:00:00.0000000', '2022-04-30T00:00:00.0000000',
       '2022-05-31T00:00:00.0000000', '2022-06-30T00:00:00.0000000',
       '2022-07-31T00:00:00.0000000', '2022-08-31T00:00:00.0000000',
       '2022-09-30T00:00:00.0000000', '2022-10-31T00:00:00.0000000',
       '2022-11-30T00:00:00.0000000', '2022-12-31T00:00:00.0000000'],
      dtype=object)

In [23]:
# Filtrer les lignes ayant une date ultérieure à la date d'aujourd'hui
today = datetime.today().strftime('%Y-%m-%d')

date_series = df_dep['period_date'].apply(lambda x: x.split('T')[0])
df_dep['format_date'] = pd.to_datetime(date_series)

df_dep = df_dep[df_dep['format_date'] <= today]
df_dep = df_dep.drop('format_date', axis=1)

In [24]:
df_dep["departement"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[0].strip())
df_dep["mesure"] = df_dep["tree_node_name"].apply(lambda x: x.split("/")[1].strip())

# nettoyage de la colonne mesure, on enlève un point surnuméraire.
df_dep["mesure"].replace("\.", "", regex=True,inplace=True)
df_dep.mesure = df_dep.mesure.apply(lambda x: clean_str(x))

df_dep.rename(columns={"effect_id":"indicateur"}, inplace=True)
df_dep.indicateur = df_dep.indicateur.str.strip()
# traduit les mois dans la colonne date
df_dep.Date = df_dep.Date.replace(month_dict, regex=True)

df_dep.valeur.replace({",": "."}, regex=True, inplace=True)
df_dep.valeur = df_dep.valeur.astype(float)

In [25]:
df_dep["short_indic"] = df_dep.indicateur.apply(lambda x: x.split("-")[0].strip())
df_dep.short_indic = df_dep.short_indic.apply(lambda x: clean_str(x))

In [26]:
def check_single_sum_dep_equals_nat_2(df_copy, df_nat, mesure, indic):
    nat_values = df_nat[(df_nat['mesure'] == mesure) &
                    (df_nat['effect_id'] == indic)][['period_date', 'valeur']].set_index('period_date').to_dict()['valeur']

    dep_indic_values = df_copy[(df_copy['mesure'] == mesure) & 
                                (df_copy['indicateur'] == indic)].groupby('period_date').sum().to_dict()['valeur']

    for date in dep_indic_values.keys():
        if date not in nat_values:
            #print(f"-- National pas de {date}")
            continue
        dep_val = round(dep_indic_values[date], 2)
        nat_val = round(nat_values[date], 2)
        #assert dep_val == nat_val, f"Somme départementale : {dep_val} - Valeur récupérée nationale : {nat_val}\n{date} - {mesure} - {indic}"
        # TODO : subtituer le if-else par un assert quand on aura la confirmation que les données sont cohérentes
        if dep_val == nat_val:
            pass
            #print(f'ok - {mesure} - {indic} - {date}')
        else:
            print(f'KO - {mesure} - {indic} - {date} | {dep_val} - {nat_val}')


def check_sum_dep_equals_nat_2(df_dep, df):
    df_nat = df.loc[(df.structure_name == "Mesure") & 
                (~df.period_month_tri.isin(forbidden_period_value)) &
                (df.state_id == 'Valeur Actuelle') &
                (~df.valeur.isna())].copy()

    df_nat["mesure"] = df_nat["tree_node_name"].apply(lambda x: clean_mesure_name(x))

    mesures = df_dep['mesure'].unique()
    for mesure in mesures:
        indics = df_dep[df_dep['mesure'] == mesure]['indicateur'].unique()
        for indic in indics:
            check_single_sum_dep_equals_nat_2(df_dep, df_nat, mesure, indic)
            
            
check_sum_dep_equals_nat_2(df_dep, df)

KO - Repas à 1 € dans les restaurants universitaires - Nombre de repas servis dans les restaurants universitaires au tarif à 1€ - RRU - 2021-01-31T00:00:00.0000000 | 3787789.0 - 3490110.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-01-31T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-02-28T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Nombre de PME - BPI - 2021-03-31T00:00:00.0000000 | 299.0 - 434.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-01-31T00:00:00.0000000 | 1277884181.0 - 1284896505.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-02-28T00:00:00.0000000 | 1277884181.0 - 1284896505.0
KO - Modernisation de la filière (BPI) - Montant cumulé de l’investissement total ainsi déclenché - BPI - 2021-03-31T00:00:00.0000000 | 1277884181.0 - 1284896505.0

In [27]:
dict_indicateur = {'Nombre de repas servis dans les restaurants universitaires au tarif à 1€' : "Nombre de repas servis",
 'Montant cumulé de l’investissement total ainsi déclenché' : "Montant cumumé de l'investissement total",
 'Nombre d’entreprises bénéficiaires du dispositif' : "Nombre d'entreprises bénéficiaires",
 'Nombre de bâtiments Etat dont le marché de rénovation est notifié' : 'Nombre de bâtiments dont le marché de rénovation est notifié',
 'Nombre de projets incluant une transformation de la ligne de production pour réduire son impact environnemental' : 'Nombre de projets',
 'Nombre d\'exploitations certifiées "haute valeur environnementale"' : 'Nombre d’exploitations certifiées',
 'Emissions de gaz à effet de serre évitées sur la durée de vie des équipements' : 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques et hybrides rechargeables' : 'Nombre de bonus octroyés à des véhicules électriques',
 "Quantité de matières plastiques évitées ou dont le recyclage ou l'intégration a été soutenue" : 'Quantité de matières plastiques évitées',
 'Montant total des travaux associés aux dossiers validés' : 'Montant total des travaux',
 'Nombre de nouveaux projets (nouvelle ligne, extension de ligne et pôle)' : 'Nombre de nouveaux projets',
 'Montant de l’investissement total déclenché' : 'Montant de l’investissement total',
 'Nombre de projets de tourisme durable financés' : 'Nombre de projets de tourisme durable financés',
 'Nombre de projets de rénovation de cathédrales et de monuments nationaux initiés' : 'Nombre de projets de rénovation',
 'Montant total investi pour la rénovation de monuments historiques appartenant aux collectivités territoriales' :'Montant total investi pour la rénovation',
 'Nombre de projets de rénovation de monuments historiques appartenant aux collectivités territoriales bénéficiaires initiés' : 'Nombre de projets de rénovation',
 "Nombre de contrats de professionnalisation bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats de professionnalisation',
 "Nombre de contrats d'apprentissage bénéficiaires de l'aide exceptionnelle" : 'Nombre de contrats d’apprentissage',
 "Nombre d'aides à l'embauche des travailleurs handicapés": "Nombre d'aides à l'embauche des travailleurs handicapés",
 'Nombre de projets locaux soutenus  (rénovation, extension, création de lignes' :'Nombre de projets locaux soutenus',
  'Nombre de dossiers MaPrimeRénov bénéficiaires ("particulier" et "copropriété")': 'Nombre de bénéficiaires',
 "Nombre de dossiers MaPrimeRénov' payés": "Nombre de dossiers payés",
  "Nombre d'entreprises bénéficiares": "Nombre d'entreprises bénéficiaires"
                  }

In [28]:
df_dep.short_indic = df_dep.short_indic.apply(lambda x: dict_indicateur[x] if x in dict_indicateur else x)

In [29]:
[x for x in list(df_dep.short_indic.unique()) if len(x) > 40]

["Montant cumulé de l'investissement total ainsi déclenché",
 "Nombre d'entreprises bénéficiaires du dispositif",
 'Nombre de bâtiments dont le marché de rénovation est notifié',
 "Montant total de l'investissement ainsi déclenché",
 'Emissions de gaz à effet de serre évitées',
 'Nombre de bonus octroyés à des véhicules électriques',
 "Montant de l'investissement total déclenché",
 'Nombre de projets de tourisme durable financés',
 "Nombre d'entreprises aidées dans les secteurs du cinéma, de l'audiovisuel, du numérique et du livre",
 'Nombre de contrats de professionnalisation',
 "Nombre d'aides à l'embauche des travailleurs handicapés"]

In [30]:
dict_mesures  = {
 "Appels à projets pour les projets d'efficacité énergétique de grande ampleur et appels à manifestation d'intérêt sur les projets qui proposent une évolution profonde des procédés industriels": "AAP Efficacité énergétique",
 "CIE jeunes" : "Contrats Initiatives Emploi (CIE) Jeunes",
 'France Num' : 'France Num : aide à la numérisation des TPE,PME,ETI',
 'Guichet efficacité énergétique dans industrie' : 'Guichet efficacité énergétique',
 "Modernisation de la filière (BPI)" : "AAP industrie : Modernisation des filières auto et aéro",
 "PEC jeunes": "Parcours emploi compétences (PEC) Jeunes",
 'Relocalisation : soutien aux projets industriels dans les territoires' : 'AAP Industrie : Soutien aux projets industriels territoires',
 'Relocalisation : sécurisation des approvisionnements critiques' : 'AAP Industrie : Sécurisation approvisionnements critiques',
 'Renforcement des subventions de Business France (chèque export, chèque VIE)' : 'Renforcement subventions Business France',
 "Soutien à la modernisation industrielle et renforcement des compétences dans la filière nucléaire" : "AAP industrie : modernisation industrielle et renforcement des compétences dans la filière nucléaire",
 'Soutien à la recherche aéronautique civil' :'Soutien recherche aéronautique civil',
 'Rénovation bâtiments Etats' : 'Rénovation des bâtiments Etats (marchés notifiés)'
}

In [31]:
df_dep["short_mesure"] = df_dep.mesure.apply(lambda x: dict_mesures[x] if x in dict_mesures else x)

In [32]:
def check_dep_coherence(df):
    assert (df['dep_code'] != df['dep']).sum() == 0, "Incoherence des codes départementaux - voir colonne dep_code et dep."
    assert df['dep'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code département."
    
    deps_test = sorted(df['dep'].unique())
    deps_true = sorted(taxo_dep_df['dep'].unique())
    assert deps_test == deps_true, "Départements ne concordent pas."
    
    assert sorted(df['libelle'].unique()) == sorted(taxo_dep_df['libelle'])
    
    
def check_reg_coherence(df):
    assert df['reg'].isnull().sum() == 0, "Certaines lignes ne possèdent pas de code région."
    
    regs_test = sorted(df['reg'].unique())
    regs_true = sorted(taxo_reg_df['reg'].unique())
    assert regs_test == regs_true, "Régions ne concordent pas."
    
    assert sorted(df['nccenr'].unique()) == sorted(taxo_reg_df['nccenr'])
    
    
def check_df_dep_enr_format(df_dep_enr):
    assert df_dep_enr.isnull().sum().sum() == 0
    assert df_dep_enr.shape[0] == df_dep.shape[0]

In [33]:
df_dep_enr = df_dep.merge(taxo_dep_df[["dep", "reg", "libelle"]], 
                          how="left", left_on="dep_code", right_on="dep")\
                   .merge(taxo_reg_df[["reg", "nccenr"]], how="left", left_on="reg", right_on="reg")
df_dep_enr.drop(columns=["tree_node_name", "structure_name"], inplace=True)

check_dep_coherence(df_dep_enr)
check_reg_coherence(df_dep_enr)

df_dep_enr.rename(columns={"nccenr":"region"}, inplace=True)
df_dep_enr.drop(['dep_code'], axis=1, inplace=True)  # Supprime code dep utilisé pour la jointure

check_df_dep_enr_format(df_dep_enr)
df_dep_enr.shape

(22644, 14)

In [ ]:
df_dep_enr.to_csv("pp_dep.csv", sep=";")